# Making dictionary

## Memory Information

In [1]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.57GB
Used: 636.82MB
Percentage: 3.7%


## GPU Information

In [2]:
! nvidia-smi

Wed Sep 30 12:55:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [6]:
file_path = '/gdrive/My Drive/ aiffel/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력
lines.info()

전체 샘플의 수 : 178009
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178009 entries, 0 to 178008
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   eng     178009 non-null  object
 1   fra     178009 non-null  object
 2   cc      178009 non-null  object
dtypes: object(3)
memory usage: 4.1+ MB


In [7]:
lines = lines[['eng', 'fra']][:33000]
lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   eng     33000 non-null  object
 1   fra     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


## 전처리

In [8]:
lines['eng'] = lines['eng'].str.lower()
lines['fra'] = lines['fra'].str.lower()

In [9]:
def preprocess_sentence(sentence) :
    
    # 전처리 부분

    # 구두점을 단어와 분리를 시켜본다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-zA-Z!.?]+", r" ", sentence)
    sentence = re.sub(r"\s+", r" ", sentence)
#     sentence = re.sub(r'[" "]+', " ", sentence)
    
    # 모두 소문자로 변환
    sentence = sentence.lower()
    
    return sentence

lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence(x))
lines.sample(5)

,eng,fra
6149,it was boring .,c tait barbant .
926,stay calm .,reste tranquille .
8601,i ve been seen .,j ai t vu .
25005,i m coming at once .,j arrive tout de suite .
10147,ask me tomorrow .,demande moi demain .


In [10]:
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : sos_token + ' ' + x + ' ' + eos_token)

In [11]:
eng_tokenizer = Tokenizer(filters="", lower=False)            # 토큰화 수행 : 문자 단위 X
eng_tokenizer.fit_on_texts(lines.eng)   # 33000개의 데이터 각 행을 토큰화
input_text = eng_tokenizer.texts_to_sequences(lines.eng)   # 단어를 숫자값 인덱스로 변환
input_text[:3]

[[30, 1], [1132, 1], [1132, 1]]

In [12]:
fra_tokenizer = Tokenizer(filters="", lower=False)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[1, 91, 12, 2], [1, 1068, 12, 2], [1, 1068, 3, 2]]

In [13]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326


In [14]:
encoder_input = input_text

# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [15]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 91, 12], [1, 1068, 12], [1, 1068, 3]]
[[91, 12, 2], [1068, 12, 2], [1068, 3, 2]]


In [16]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [17]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [18]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [19]:
print(decoder_input[0])

[ 1 91 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [20]:
eng_to_index = eng_tokenizer.word_index
index_to_eng = eng_tokenizer.index_word

fra_to_index = fra_tokenizer.word_index
index_to_fra = fra_tokenizer.index_word

## 데이터셋 나누기

In [21]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[22855 22810 12539 ... 27680 31492 26604]


In [22]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [23]:
encoder_input_train = encoder_input[:30000]
decoder_input_train = decoder_input[:30000]
decoder_target_train = decoder_target[:30000]

encoder_input_test = encoder_input[30000:]
decoder_input_test = decoder_input[30000:]
decoder_target_test = decoder_target[30000:]

In [24]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


## 모델 훈련

In [25]:
latent_dim = 32

In [26]:
# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, latent_dim)(encoder_inputs) # 임베딩층
enc_masking = Masking(mask_value=0.0)(enc_emb)  # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True)  # 상태값 리턴
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)  # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]  # 인코더의 은닉 상태외 셀 상태를 저장

In [27]:
# 디코더
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(fra_vocab_size, latent_dim)  # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)  # 패딩 0은 언제나 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequence는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태 (initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점에 결과에 대해 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 32)     149184      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     234432      input_2[0][0]                    
_______________________________________________________________________________________

In [29]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
235/235 [==============================] - 13s 55ms/step - loss: 3.8871 - acc: 0.6104 - val_loss: 2.1126 - val_acc: 0.6099
Epoch 2/50
235/235 [==============================] - 10s 42ms/step - loss: 1.8501 - acc: 0.6844 - val_loss: 1.7091 - val_acc: 0.7321
Epoch 3/50
235/235 [==============================] - 10s 42ms/step - loss: 1.6191 - acc: 0.7385 - val_loss: 1.5785 - val_acc: 0.7432
Epoch 4/50
235/235 [==============================] - 10s 42ms/step - loss: 1.5191 - acc: 0.7496 - val_loss: 1.5049 - val_acc: 0.7523
Epoch 5/50
235/235 [==============================] - 10s 42ms/step - loss: 1.4467 - acc: 0.7590 - val_loss: 1.4407 - val_acc: 0.7653
Epoch 6/50
235/235 [==============================] - 10s 42ms/step - loss: 1.3835 - acc: 0.7733 - val_loss: 1.3841 - val_acc: 0.7766
Epoch 7/50
235/235 [==============================] - 10s 42ms/step - loss: 1.3298 - acc: 0.7836 - val_loss: 1.3382 - val_acc: 0.7864
Epoch 8/50
235/235 [==============================] - 10s 42ms

## 모델 테스트

In [30]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 32)          149184    
_________________________________________________________________
masking (Masking)            (None, None, 32)          0         
_________________________________________________________________
lstm (LSTM)                  [(None, 32), (None, 32),  8320      
Total params: 157,504
Trainable params: 157,504
Non-trainable params: 0
_________________________________________________________________


In [31]:
# 디코더 설계
# 이전 시점의 상태롤 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# train 때 사용했던 임베딩 층을 재사용..
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단여 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [32]:
# 디코더 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [33]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_fra[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [34]:
def seq2eng(input_seq):
    temp = ''
    for i in input_seq :
        if(i!=0):
            temp = temp + index_to_eng[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2fra(input_seq):
    temp = ''
    for i in input_seq:
        if ((i!=0 and i!=fra_to_index['<sos>']) and i!=fra_to_index['<eos>']):
            temp = temp + index_to_fra[i] + ' '
    return temp

In [35]:
for seq_index in [3, 50, 100, 600, 2005]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("원문 : ", seq2eng(encoder_input_train[seq_index]))
    print("번역문 : ", seq2fra(decoder_input_train[seq_index]))
    print("예측문 : ", decoded_sentence[:-5])
    print('\n')

원문 :  should i start ? 
번역문 :  devrais je commencer ? 
예측문 :   devrais je il manqu ? 


원문 :  stay close . 
번역문 :  reste pr s . 
예측문 :   restez en che ! 


원문 :  i think tom is hurt . 
번역문 :  je pense que tom est bless . 
예측문 :   je pense que tom est qu il est malade . 


원문 :  be more precise . 
번역문 :  soyez plus pr cis . 
예측문 :   pr sol suis ! 


원문 :  i survived . 
번역문 :  j ai surv cu . 
예측문 :   j ai eu . 




## 정리

처음엔 33000개의 RAM이 문제인줄 알았는데 내가 잘못 생각하면서 불필요한 RAM생산을 한 것이었다. 좀더 세심한 코드 공부가 필요할 것 같다.     
그리고 이번에는 COLAB에서 직접 했는데 나름 성과가 있었다. COLAB 이용을 자주 해야겠다는 생각이 들었다.